# Form the NB Regression Random Intercept Model: Simulated set

Next we may try joint estimation using Newton's + IPOPT. 

    I create new structures `NBCopulaVCObs`, `NBCopulaVCModel`
    



In [1]:
using DataFrames, Random, GLM, GLMCopula, Revise
using ForwardDiff, Test, LinearAlgebra
using LinearAlgebra: BlasReal, copytri!

Random.seed!(1234)

# sample size
N = 10000
# observations per subject
n = 5

variance_component_1 = 0.1
variance_component_2 = 0.1

r = 1
p = 0.7
μ = r * (1-p) * inv(p)

# var = r * (1-p) * inv(p^2)

# true beta
β_true = log(μ)

dist = NegativeBinomial

Γ = variance_component_1 * ones(n, n) + variance_component_2 * Matrix(I, n, n)
vecd = [dist(r, p) for i in 1:n]
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)

Y_Nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, N)

10000-element Array{Array{Float64,1},1}:
 [0.0, 1.0, 0.0, 0.0, 1.0]
 [1.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 3.0, 1.0]
 [0.0, 0.0, 0.0, 0.0, 1.0]
 [3.0, 3.0, 2.0, 1.0, 1.0]
 [0.0, 0.0, 0.0, 0.0, 0.0]
 [2.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 1.0, 0.0, 0.0, 1.0]
 [0.0, 1.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 1.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0]
 ⋮
 [0.0, 0.0, 1.0, 0.0, 0.0]
 [1.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 1.0, 0.0, 0.0, 1.0]
 [0.0, 0.0, 1.0, 1.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 2.0]
 [0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 1.0, 1.0, 1.0]
 [1.0, 2.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 1.0, 0.0]
 [2.0, 0.0, 1.0, 0.0, 0.0]
 [0.0, 0.0, 1.0, 0.0, 0.0]

In [2]:
d = NegativeBinomial()
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{NBCopulaVCObs{T, D, Link}}(undef, N)
for i in 1:N
    y = Float64.(Y_Nsample[i])
    X = ones(n, 1)
    V = [ones(n, n), Matrix(I, n, n)]
    gcs[i] = NBCopulaVCObs(y, X, V, d, link)
end
gcm = NBCopulaVCModel(gcs);

In [3]:
initialize_model!(gcm)
@show gcm.β
@show gcm.Σ

initializing β using Newton's Algorithm under Independence Assumption
1 0.0 -50596.55142422627 39999
2 -50596.55142422627 -50596.55142422627 39999
initializing variance components using MM-Algorithm
gcm.β = [-0.5911322551532158]
gcm.Σ = [0.046176044902570046, 0.0018338952408204375]


2-element Array{Float64,1}:
 0.046176044902570046
 0.0018338952408204375

In [4]:
β_true

-0.8472978603872034

Initialize β and σ2, here I just copy the solution for β and σ2 from MixedModels.jl over

In [5]:
GLMCopula.loglikelihood!(gcm, true, true)

-50482.63010459834

## Try Quasi-Newton after specifying gradient of r

to-do: 

1. added field $r$ to model structure $gcm.r$, change access to r this way instead of through the distribution
2. add $gc.\nabla r$ in loglikelihood function


## Newton after specifying the hessian term too

3. add $gc.Hr$ in loglikelihood function

In [6]:
# Quasi-Newton
# @time GLMCopula.fit!(gcm, IpoptSolver(print_level = 5, max_iter = 100, hessian_approximation = "limited-memory"))

In [7]:
# @show β_true
# @show gcm.β
# @show gcm.Σ
# @show gcm.∇β
# @show GLMCopula.loglikelihood!(gcm, true, true);

In [8]:
# println("estimated β = $(gcm.β[1]); true β = $β_true")
# println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
# println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2");

In [9]:
# Newton
# @time GLMCopula.fit!(gcm, IpoptSolver(print_level = 5, max_iter = 100, hessian_approximation = "exact"))